In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sstats 
from statsmodels.stats import contingency_tables
from os.path import exists, join

In [2]:
PROJ_DIR = '/Volumes/Projects_Herting/LABDOCS/Personnel/Katie/deltaABCD_clustering'
DATA_DIR = 'data'
FIG_DIR = 'figures'

In [3]:
df = pd.read_csv(join(PROJ_DIR, DATA_DIR, 'data_qcd.csv'), 
                 header=0, 
                 index_col='subjectkey')

In [4]:
demographics = [#"demo_prnt_ethn_v2",
                "demo_prnt_marital_v2",
                "demo_prnt_ed_v2",
                "demo_comb_income_v2",
                #"demo_race_a_p___10",
                #"demo_race_a_p___11",
                #"demo_race_a_p___12",
                #"demo_race_a_p___13",
                #"demo_race_a_p___14",
                #"demo_race_a_p___15",
                #"demo_race_a_p___16",
                #"demo_race_a_p___17",
                #"demo_race_a_p___18",
                #"demo_race_a_p___19",
                #"demo_race_a_p___20",
                #"demo_race_a_p___21",
                #"demo_race_a_p___22",
                #"demo_race_a_p___23",
                #"demo_race_a_p___24",
                #"demo_race_a_p___25",
                "race_ethnicity",
                "site_id_l",
                "sex", 
                "mri_info_manufacturer"
               ]
mri_qc = [
    "imgincl_dmri_include",
    "imgincl_rsfmri_include",
    "imgincl_t1w_include",
    #"imgincl_t2w_include",
    "mrif_score",
    "interview_age",
    "interview_date"
]

In [5]:
demo_and_qc = []
for var in demographics + mri_qc:
    demo_and_qc.append(f'{var}.baseline_year_1_arm_1')
    if var in mri_qc:
        demo_and_qc.append(f'{var}.2_year_follow_up_y_arm_1')
    else:
        pass

In [6]:
demo_and_qc

['demo_prnt_marital_v2.baseline_year_1_arm_1',
 'demo_prnt_ed_v2.baseline_year_1_arm_1',
 'demo_comb_income_v2.baseline_year_1_arm_1',
 'race_ethnicity.baseline_year_1_arm_1',
 'site_id_l.baseline_year_1_arm_1',
 'sex.baseline_year_1_arm_1',
 'mri_info_manufacturer.baseline_year_1_arm_1',
 'imgincl_dmri_include.baseline_year_1_arm_1',
 'imgincl_dmri_include.2_year_follow_up_y_arm_1',
 'imgincl_rsfmri_include.baseline_year_1_arm_1',
 'imgincl_rsfmri_include.2_year_follow_up_y_arm_1',
 'imgincl_t1w_include.baseline_year_1_arm_1',
 'imgincl_t1w_include.2_year_follow_up_y_arm_1',
 'mrif_score.baseline_year_1_arm_1',
 'mrif_score.2_year_follow_up_y_arm_1',
 'interview_age.baseline_year_1_arm_1',
 'interview_age.2_year_follow_up_y_arm_1',
 'interview_date.baseline_year_1_arm_1',
 'interview_date.2_year_follow_up_y_arm_1']

In [7]:
demo_df = df[demo_and_qc]

#site_baseline = pd.get_dummies(demo_df, 'site_id_l.baseline_year_1_arm_1')
#site_2yfu = pd.get_dummies(demo_df, 'site_id_l.2_year_follow_up_y_arm_1')

#demo_df = pd.concat([demo_df, site_baseline, site_2yfu], axis=1)

In [8]:
df = None

In [9]:
no_2yfu = demo_df[demo_df["interview_date.2_year_follow_up_y_arm_1"].isna() == True].index
lost_N = len(no_2yfu)
total_N = len(demo_df.index)

y2fu_df = demo_df.drop(no_2yfu, axis=0)

In [10]:
print(f"Of the total {total_N} participants at baseline, {lost_N} (or {np.round((lost_N / total_N) *100, 2)}%) did not have a 2-year follow-up imaging appointment and were, thus, excluded from further analyses.")

Of the total 7457 participants at baseline, 0 (or 0.0%) did not have a 2-year follow-up imaging appointment and were, thus, excluded from further analyses.


In [11]:
table = pd.DataFrame(index=['N', 
                            'Age_mean_base',
                            'Age_sdev_base',
                            'Age_mean_2yfu',
                            'Age_sdev_2yfu',
                            'Sex_M', 
                            'Sex_F', 
                            'RE_Black',
                            'RE_White',
                            'RE_Hispanic',
                            'RE_Asian',
                            'RE_Other',
                            'Income_gt100k', 
                            'Income_50to100k', 
                            'Income_lt50k',
                            'Income_dkrefuse',
                            'Marital_Married',
                            'Marital_Widowed',
                            'Marital_Divorced',
                            'Marital_Separated',
                            'Marital_Never',
                            'Marital_Refused',
                            'Education_ltHS',
                            'Education_HSGED',
                            'Education_SomeCol',
                            'Education_AA',
                            'Education_Bachelors',
                            'Education_Graduate',
                            'MRI_Siemens', 
                            'MRI_GE', 
                            'MRI_Phillips'], 
                     columns=['whole_sample', 'with_2yfu'])

table.at['N', 'whole_sample'] = len(demo_df.index)
table.at['N', 'with_2yfu'] = len(y2fu_df.index)

table.at['Age_mean_base', 'whole_sample'] = np.mean(demo_df['interview_age.baseline_year_1_arm_1'])
table.at['Age_mean_base', 'with_2yfu'] = np.mean(y2fu_df['interview_age.baseline_year_1_arm_1'])

table.at['Age_mean_2yfu', 'whole_sample'] = np.mean(demo_df['interview_age.2_year_follow_up_y_arm_1'])
table.at['Age_mean_2yfu', 'with_2yfu'] = np.mean(y2fu_df['interview_age.2_year_follow_up_y_arm_1'])

table.at['Age_sdev_base', 'whole_sample'] = np.std(demo_df['interview_age.baseline_year_1_arm_1'])
table.at['Age_sdev_base', 'with_2yfu'] = np.std(y2fu_df['interview_age.baseline_year_1_arm_1'])

table.at['Age_sdev_2yfu', 'whole_sample'] = np.std(demo_df['interview_age.2_year_follow_up_y_arm_1'])
table.at['Age_sdev_2yfu', 'with_2yfu'] = np.std(y2fu_df['interview_age.2_year_follow_up_y_arm_1'])

table.at['Sex_M', 'whole_sample'] = len(demo_df[demo_df['sex.baseline_year_1_arm_1'] == 'M'].index)
table.at['Sex_M', 'with_2yfu'] = len(y2fu_df[y2fu_df['sex.baseline_year_1_arm_1'] == 'M'].index)
table.at['Sex_F', 'whole_sample'] = len(demo_df[demo_df['sex.baseline_year_1_arm_1'] == 'F'].index)
table.at['Sex_F', 'with_2yfu'] = len(y2fu_df[y2fu_df['sex.baseline_year_1_arm_1'] == 'F'].index)


table.at['RE_White', 
         'whole_sample'] = len(demo_df[demo_df['race_ethnicity.baseline_year_1_arm_1'] == 1.].index)
table.at['RE_White', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['race_ethnicity.baseline_year_1_arm_1'] == 1.].index)
table.at['RE_Black', 
         'whole_sample'] = len(demo_df[demo_df['race_ethnicity.baseline_year_1_arm_1'] == 2.].index)
table.at['RE_Black', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['race_ethnicity.baseline_year_1_arm_1'] == 2.].index)
table.at['RE_Hispanic', 
         'whole_sample'] = len(demo_df[demo_df['race_ethnicity.baseline_year_1_arm_1'] == 3.].index)
table.at['RE_Hispanic', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['race_ethnicity.baseline_year_1_arm_1'] == 3.].index)
table.at['RE_AsianOther', 
         'whole_sample'] = len(demo_df[demo_df['race_ethnicity.baseline_year_1_arm_1'].between(4.,5.,inclusive='both')].index)
table.at['RE_AsianOther', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['race_ethnicity.baseline_year_1_arm_1'].between(4.,5.,inclusive='both')].index)


table.at['Income_gt100k', 
         'whole_sample'] = len(demo_df[demo_df['demo_comb_income_v2.baseline_year_1_arm_1'].between(9.,10., inclusive='both')].index)
table.at['Income_gt100k', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['demo_comb_income_v2.baseline_year_1_arm_1'].between(9.,10., inclusive='both')].index)

table.at['Income_50to100k', 
         'whole_sample'] = len(demo_df[demo_df['demo_comb_income_v2.baseline_year_1_arm_1'].between(7., 8., inclusive='both')].index)
table.at['Income_50to100k', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['demo_comb_income_v2.baseline_year_1_arm_1'].between(7., 8., inclusive='both')].index)

table.at['Income_lt50k', 
         'whole_sample'] = len(demo_df[demo_df['demo_comb_income_v2.baseline_year_1_arm_1'] <= 6.].index)
table.at['Income_lt50k', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['demo_comb_income_v2.baseline_year_1_arm_1'] <= 6.].index)

table.at['Income_dkrefuse', 
         'whole_sample'] = len(demo_df[demo_df['demo_comb_income_v2.baseline_year_1_arm_1'] >= 777.].index)
table.at['Income_dkrefuse', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['demo_comb_income_v2.baseline_year_1_arm_1'] >= 777.].index)

table.at['MRI_Siemens', 
         'whole_sample'] = len(demo_df[demo_df['mri_info_manufacturer.baseline_year_1_arm_1'] == "SIEMENS"].index)
table.at['MRI_Siemens', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['mri_info_manufacturer.baseline_year_1_arm_1'] == "SIEMENS"].index)
table.at['MRI_GE', 
         'whole_sample'] = len(demo_df[demo_df['mri_info_manufacturer.baseline_year_1_arm_1'] == "GE MEDICAL SYSTEMS"].index)
table.at['MRI_GE', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['mri_info_manufacturer.baseline_year_1_arm_1']  == "GE MEDICAL SYSTEMS"].index)
table.at['MRI_Philips', 
         'whole_sample'] = len(demo_df[demo_df['mri_info_manufacturer.baseline_year_1_arm_1'] == "Philips Medical Systems"].index)
table.at['MRI_Philips', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['mri_info_manufacturer.baseline_year_1_arm_1'] == "Philips Medical Systems"].index)

table.at['Marital_Married', 
         'whole_sample'] = len(demo_df[demo_df["demo_prnt_marital_v2.baseline_year_1_arm_1"] == 1.])
table.at['Marital_Married', 
         'with_2yfu'] = len(y2fu_df[y2fu_df["demo_prnt_marital_v2.baseline_year_1_arm_1"] == 1.])
table.at['Marital_Widowed', 
         'whole_sample'] = len(demo_df[demo_df["demo_prnt_marital_v2.baseline_year_1_arm_1"] == 2.])
table.at['Marital_Widowed', 
         'with_2yfu'] = len(y2fu_df[y2fu_df["demo_prnt_marital_v2.baseline_year_1_arm_1"] == 2.])
table.at['Marital_Divorced', 
         'whole_sample'] = len(demo_df[demo_df["demo_prnt_marital_v2.baseline_year_1_arm_1"] == 3.])
table.at['Marital_Divorced', 
         'with_2yfu'] = len(y2fu_df[y2fu_df["demo_prnt_marital_v2.baseline_year_1_arm_1"] == 3.])
table.at['Marital_Separated', 
         'whole_sample'] = len(demo_df[demo_df["demo_prnt_marital_v2.baseline_year_1_arm_1"] == 4.])
table.at['Marital_Separated', 
         'with_2yfu'] = len(y2fu_df[y2fu_df["demo_prnt_marital_v2.baseline_year_1_arm_1"] == 4.])
table.at['Marital_Never', 
         'whole_sample'] = len(demo_df[demo_df["demo_prnt_marital_v2.baseline_year_1_arm_1"] == 5.])
table.at['Marital_Never', 
         'with_2yfu'] = len(y2fu_df[y2fu_df["demo_prnt_marital_v2.baseline_year_1_arm_1"] == 5.])
table.at['Marital_Refused', 
         'whole_sample'] = len(demo_df[demo_df["demo_prnt_marital_v2.baseline_year_1_arm_1"] == 777.])
table.at['Marital_Refused', 
         'with_2yfu'] = len(y2fu_df[y2fu_df["demo_prnt_marital_v2.baseline_year_1_arm_1"] == 777.])

table.at['Education_uptoHSGED', 
         'whole_sample'] = len(demo_df[demo_df['demo_prnt_ed_v2.baseline_year_1_arm_1'].between(0,14, 
                                                                                                inclusive='both')])
table.at['Education_SomeColAA', 
         'whole_sample'] = len(demo_df[demo_df['demo_prnt_ed_v2.baseline_year_1_arm_1'].between(15,17, 
                                                                                                inclusive='both')])
table.at['Education_Bachelors', 
         'whole_sample'] = len(demo_df[demo_df['demo_prnt_ed_v2.baseline_year_1_arm_1'] == 18])
table.at['Education_Graduate', 
         'whole_sample'] = len(demo_df[demo_df['demo_prnt_ed_v2.baseline_year_1_arm_1'].between(19,22, 
                                                                                                inclusive='both')])
table.at['Education_uptoHSGED', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['demo_prnt_ed_v2.baseline_year_1_arm_1'].between(0,14, 
                                                                                                inclusive='both')])
table.at['Education_SomeColAA', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['demo_prnt_ed_v2.baseline_year_1_arm_1'].between(15,17, 
                                                                                                inclusive='both')])
table.at['Education_Bachelors', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['demo_prnt_ed_v2.baseline_year_1_arm_1'] == 18])
table.at['Education_Graduate', 
         'with_2yfu'] = len(y2fu_df[y2fu_df['demo_prnt_ed_v2.baseline_year_1_arm_1'].between(19,22, 
                                                                                                inclusive='both')])

In [13]:
# test for differences in means with wilcoxon signed rank test
stat, pval = sstats.mannwhitneyu(demo_df['interview_age.baseline_year_1_arm_1'].dropna(), 
                                 y2fu_df['interview_age.baseline_year_1_arm_1'].dropna())
table.at['Age_mean_base', 'Stat'] = stat
table.at['Age_mean_base', 'p(Stat)'] = pval

stat, pval = sstats.mannwhitneyu(demo_df['interview_age.2_year_follow_up_y_arm_1'].dropna(), 
                                 y2fu_df['interview_age.2_year_follow_up_y_arm_1'].dropna())
table.at['Age_mean_2yfu', 'Stat'] = stat
table.at['Age_mean_2yfu', 'p(Stat)'] = pval

contingency = np.zeros((2,2))
contingency[0,0] = table.loc['Sex_M', 'whole_sample']
contingency[0,1] = table.loc['Sex_F', 'whole_sample']
contingency[1,0] = table.loc['Sex_M', 'with_2yfu']
contingency[1,1] = table.loc['Sex_F', 'with_2yfu']
out = contingency_tables.mcnemar(contingency) 
table.at['Sex_M', 'Stat'] = out.statistic
table.at['Sex_M', 'p(Stat)'] = out.pvalue


contingency = np.zeros((2,5))
contingency[0,0] = table.loc['RE_White', 'whole_sample']
contingency[0,1] = table.loc['RE_Black', 'whole_sample']
contingency[0,2] = table.loc['RE_Hispanic', 'whole_sample']
contingency[0,3] = table.loc['RE_Asian', 'whole_sample']
contingency[0,4] = table.loc['RE_Other', 'whole_sample']
contingency[1,0] = table.loc['RE_White', 'with_2yfu']
contingency[1,1] = table.loc['RE_Black', 'with_2yfu']
contingency[1,2] = table.loc['RE_Hispanic', 'with_2yfu']
contingency[1,3] = table.loc['RE_Asian', 'with_2yfu']
contingency[1,4] = table.loc['RE_Other', 'with_2yfu']
out = contingency_tables.cochrans_q(contingency) 
table.at['RE_White', 'Stat'] = out.statistic
table.at['RE_White', 'p(Stat)'] = out.pvalue

contingency = np.zeros((2,4))
contingency[0,0] = table.loc['Income_gt100k', 'whole_sample']
contingency[0,1] = table.loc['Income_50to100k', 'whole_sample']
contingency[0,2] = table.loc['Income_lt50k', 'whole_sample']
contingency[0,3] = table.loc['Income_dkrefuse', 'whole_sample']
contingency[1,0] = table.loc['Income_gt100k', 'with_2yfu']
contingency[1,1] = table.loc['Income_50to100k', 'with_2yfu']
contingency[1,2] = table.loc['Income_lt50k', 'with_2yfu']
contingency[1,3] = table.loc['Income_dkrefuse', 'with_2yfu']
out = contingency_tables.cochrans_q(contingency) 
table.at['Income_gt100k', 'Stat'] = out.statistic
table.at['Income_gt100k', 'p(Stat)'] = out.pvalue

contingency = np.zeros((2,3))
contingency[0,0] = table.loc['MRI_Siemens', 'whole_sample']
contingency[0,1] = table.loc['MRI_GE', 'whole_sample']
contingency[0,2] = table.loc['MRI_Philips', 'whole_sample']
contingency[1,0] = table.loc['MRI_Siemens', 'with_2yfu']
contingency[1,1] = table.loc['MRI_GE', 'with_2yfu']
contingency[1,2] = table.loc['MRI_Philips', 'with_2yfu']
out = contingency_tables.cochrans_q(contingency) 
table.at['MRI_Siemens', 'Stat'] = out.statistic
table.at['MRI_Siemens', 'p(Stat)'] = out.pvalue

contingency = np.zeros((2,5))
contingency[0,0] = table.loc['Marital_Married', 'whole_sample']
contingency[0,1] = table.loc['Marital_Widowed', 'whole_sample']
contingency[0,2] = table.loc['Marital_Divorced', 'whole_sample']
contingency[0,3] = table.loc['Marital_Separated', 'whole_sample']
contingency[0,4] = table.loc['Marital_Never', 'whole_sample']
contingency[1,0] = table.loc['Marital_Married', 'with_2yfu']
contingency[1,1] = table.loc['Marital_Widowed', 'with_2yfu']
contingency[1,2] = table.loc['Marital_Divorced', 'with_2yfu']
contingency[1,3] = table.loc['Marital_Separated', 'with_2yfu']
contingency[1,4] = table.loc['Marital_Never', 'with_2yfu']
out = contingency_tables.cochrans_q(contingency) 
table.at['Marital_Married', 'Stat'] = out.statistic
table.at['Marital_Married', 'p(Stat)'] = out.pvalue

contingency = np.zeros((2,4))
contingency[0,0] = table.loc['Education_uptoHSGED', 'whole_sample']
contingency[0,1] = table.loc['Education_SomeColAA', 'whole_sample']
contingency[0,2] = table.loc['Education_Bachelors', 'whole_sample']
contingency[0,3] = table.loc['Education_Graduate', 'whole_sample']
contingency[1,0] = table.loc['Education_uptoHSGED', 'with_2yfu']
contingency[1,1] = table.loc['Education_SomeColAA', 'with_2yfu']
contingency[1,2] = table.loc['Education_Bachelors', 'with_2yfu']
contingency[1,3] = table.loc['Education_Graduate', 'with_2yfu']
out = contingency_tables.cochrans_q(contingency) 
table.at['Education_ltHS', 'Stat'] = out.statistic
table.at['Education_ltHS', 'p(Stat)'] = out.pvalue

/Users/katherine.b/Library/Python/3.8/lib/python/site-packages/statsmodels/stats/contingency_tables.py:1405: RuntimeWarning: invalid value encountered in double_scalars
  q_stat = ((k-1) * (k * np.sum(count_col_success**2) - count_col_ss**2)


In [15]:
contingency

array([[nan, nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan, nan]])

In [ ]:
table.loc['Sex_F', 'whole_sample'] / table.loc['N', 'whole_sample']

In [ ]:
table.loc['Sex_F', 'with_2yfu'] / table.loc['N', 'with_2yfu']